In [1]:
# files and kungfauxpandas loading require reference from one directory level up
import os
os.chdir('..')

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../../data/sample_data.db")
cursor = conn.cursor()

In [3]:
from kungfauxpandas import KungFauxPandas, TrivialPlugin, DataSynthesizerPlugin, KDEPlugin
kfpd = KungFauxPandas()

In [4]:
sql = '''
SELECT d."SubjectId", d."EncounterId", d."Source", d."Code", d."Type", MAX("FlowsheetValue") AS MaxScore, MIN("FlowsheetValue") AS MinScore, COUNT("FlowsheetValue") AS NumLoggedScores
 FROM diagnoses d LEFT JOIN flowsheet f ON d."EncounterId" = f."EncounterId"
 GROUP BY d."SubjectId", d."EncounterId", d."Source", d."Code", d."Type"
 ORDER BY NumLoggedScores DESC
 limit 1000
'''

In [5]:
kfpd.plugin = TrivialPlugin()
fdf=kfpd.read_sql(sql,conn)
fdf.head()

,SubjectId,EncounterId,Source,Code,Type,MaxScore,MinScore,NumLoggedScores
0,40552133,288,Encounter,A4152,ICD-10-CM,100.0,0.0,294
1,83299697,625,Encounter,A414,ICD-10-CM,100.0,0.0,286
2,96360391,985,Encounter,A400,ICD-10-CM,100.0,0.0,278
3,43551783,984,Encounter,A392,ICD-10-CM,98.0,0.0,272
4,92110570,934,Encounter,A4151,ICD-10-CM,99.0,0.0,256


In [6]:
kfpd.plugin = DataSynthesizerPlugin(mode='independent_attribute_mode', verbose=False)
fdf=kfpd.read_sql(sql,conn)
fdf.head()

Skipping read from csv and returning the input data frame


,SubjectId,EncounterId,Source,Code,Type,MaxScore,MinScore,NumLoggedScores
0,59091572.0,649.0,Problem List,uzfdu,ICD-10-CM,99.729218,0.0,140.0
1,68624515.0,98.0,Encounter,ujrtry,ICD-10-CM,99.036495,0.0,129.0
2,66943722.0,393.0,Encounter,lclylb,ICD-10-CM,95.935947,0.0,135.0
3,56174670.0,738.0,Billing,wzrbbu,ICD-10-CM,99.738615,0.0,129.0
4,52450025.0,117.0,Encounter,zuhamr,ICD-10-CM,99.080197,0.0,133.0


In [7]:
kfpd.plugin = KDEPlugin(verbose = False, mode='independent_attribute_mode')
fdf=kfpd.read_sql(sql,conn)
fdf.head()

,SubjectId,EncounterId,Source,Code,Type,MaxScore,MinScore,NumLoggedScores
0,-10308664,141,Billing,N950,ICD-10-CM,100.0,1.0,135
1,31175541,241,Problem List,A5486,ICD-10-CM,100.0,0.0,132
2,67298643,838,Encounter,A327,ICD-10-CM,99.0,0.0,137
3,49070319,807,Patient History,S45001D,ICD-10-CM,97.0,0.0,127
4,71361333,205,Patient History,Y629,ICD-10-CM,99.0,0.0,141


In [8]:
kfpd.plugin = DataSynthesizerPlugin(mode='correlated_attribute_mode',
                                    candidate_keys = {'SubjectId': True, 'EncounterId': True},
                                    categorical_attributes = {'Source': True,
                                                              'Code': True,
                                                              'Type': True,
                                                              'MaxScore': False,
                                                              'MinScore': False,
                                                              'NumLoggedScores': False}
                                   )
fdf=kfpd.read_sql(sql,conn)
fdf.head()

Skipping read from csv and returning the input data frame
================ Constructing Bayesian Network (BN) ================
Adding ROOT Code
Adding attribute NumLoggedScores
Adding attribute MaxScore
Adding attribute Type
Adding attribute Source
Adding attribute MinScore
========================= BN constructed =========================


,SubjectId,EncounterId,Source,Code,Type,MaxScore,MinScore,NumLoggedScores
0,0,0,Encounter,S61203D,ICD-10-CM,95.453327,0.0,211.0
1,1,1,Billing,S89311P,ICD-10-CM,99.027386,0.0,267.0
2,2,2,Encounter,S66291A,ICD-10-CM,98.926171,0.0,194.0
3,3,3,Encounter,S60944D,ICD-10-CM,98.577675,0.0,126.0
4,4,4,Encounter,S36299D,ICD-10-CM,99.253382,0.0,278.0
